<a href="https://colab.research.google.com/github/sualeh/introduction-to-chatgpt-api/blob/main/chatgpt-api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

----------

> **How to Run This Notebook**

To get started, create an Open AI API account, set up billing, and generate and API key at https://platform.openai.com/. If you are running the notebook locally in Visual Studio Code or other IDE, create a file called `.env`, and add a line `OPENAI_API_KEY=<your-openai-api-key>`. This key will be read by the `load_dotenv` library.

Otherwise, if you are running in Google Colab, create a secret called `OPENAI_API_KEY` and set it to the value of your OpenAI API key.

Run the code below to read the key.


In [ ]:
%pip install -qq python-dotenv

from os import environ as env
from dotenv import load_dotenv
import logging

logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

# Load key from an environmental variable called "OPENAI_API_KEY"
# Use python-dotenv https://pypi.org/project/python-dotenv/
# And take environment variables from .env
load_dotenv()
try:
  # Attempt to read OPENAI_API_KEY from a Google Colab secret
  from google.colab import userdata
  env['OPENAI_API_KEY'] = env.get('OPENAI_API_KEY', userdata.get('OPENAI_API_KEY'))
except ModuleNotFoundError:
  logger.info("Not running in Google Colab")
  # No action - rely on the OPENAI_API_KEY environmental variable



----------

# Using Retrieval Augmented Generation (RAG) with Vector Databases

Let us see how to use the vector database we created earlier as a knowledge base for OpenAI's ChatGPT model. This creates a powerful RAG (Retrieval Augmented Generation) system that can answer questions based on your local documents.

In [ ]:
# Install necessary packages
%pip install -qq langchain langchain_openai faiss-cpu python-dotenv

## Setup and Configuration

First, let's import the required libraries and set up our environment.

In [ ]:
import os
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

# Load environment variables
load_dotenv()

# Check if the API key is available
if "OPENAI_API_KEY" not in os.environ:
    openai_api_key = input("Please enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

# Initialize the OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Set default vector database path
VECTOR_DB_PATH = "./vector_db"

# Load parameter configuration
load_dotenv(dotenv_path=".env.params")
if os.getenv("VECTOR_DB_PATH"):
    VECTOR_DB_PATH = os.getenv("VECTOR_DB_PATH")

## Load Vector Database

Now let's create a function to load our previously created vector database.

In [ ]:
from langchain.vectorstores import FAISS

def load_vector_database(db_path=VECTOR_DB_PATH):
    """
    Load a FAISS vector database from the specified path.
    
    Args:
        db_path (str): Path to the vector database
        
    Returns:
        FAISS: The loaded vector database or None if not found
    """
    if not os.path.exists(db_path):
        logger.error(f"Error: Vector database not found at {db_path}")
        return None
    
    try:
        vector_db = FAISS.load_local(db_path, embeddings, allow_dangerous_deserialization=True)
        logger.info(f"Vector database successfully loaded from {db_path}")
        return vector_db
    except Exception as e:
        logger.error(f"Error loading vector database: {e}")
        return None

## Set Up the Chat Model

We need to load up the previously created vector database that contains embedding, set up the OpenAI ChatGPT model.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# Load the vector database
vector_db = load_vector_database()

# Set up the chat model, using langchain
chat_model = ChatOpenAI(
        model_name="gpt-3.5-turbo",
        temperature=0.7
    )
# Create the RAG prompt template
prompt_template = ChatPromptTemplate.from_template("""
    You are a helpful assistant that provides accurate information based on the given context.
    If you don't know the answer based on the context, just say that you don't know.
    Don't try to make up an answer.
    
    Context:
    {context}
    
    Question: {question}
    
    Answer:
    """)

# Create a retriever from the vector database
k = 3
retriever = vector_db.as_retriever(search_kwargs={"k": k})

# Format the retrieved documents into a single context string
# Also include source numbers for citation
def format_docs(docs):
    # DEBUG: Print the retrieved documents
    logger.info(f"Retrieved {len(docs)} documents:")
    for i, doc in enumerate(docs):
        source_info = f"Source [{i+1}]"
        print(doc.metadata) 
        if 'source' in doc.metadata:
            source_info = source_info + f": {doc.metadata['source']}"
        page_content = doc.page_content.replace("\n", "")
        logger.info(f"{source_info}\n\t{page_content[:50]} ... {page_content[-50:]}")

    return "\n\n".join(doc.page_content for doc in docs)

# Create the RAG chain
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | chat_model
    | StrOutputParser()
)

question = "What are the main topics covered in the most commonly referenced documents?"
print(f"\nQuestion: {question}\n\n")
answer = rag_chain.invoke(question)
print(f"\n\nAnswer: {answer}")